In [241]:
import numpy as np

In [242]:
import time

In [243]:
from numba import jit

In [244]:
from komoran3py import KomoranPy

In [272]:
from multiprocessing import Process, Pool

In [245]:
import pandas as pd

In [246]:
ko = KomoranPy()

In [247]:
ko.set_user_dictionary('./user_dictionary.txt')

In [248]:
with open('../../../data/twitch_live_chat/#zilioner/2018-12-24_#zilioner.log') as f : 
    text = f.readlines()

In [249]:
text

['# Log started: 2018-12-24 10:32:42 +0900\n',
 '[10:32:42] <@%Jeongoon> 치뫄\n',
 '[10:33:26] <~%침착맨 (zilioner)> 하이요\n',
 '[10:33:27] <~%침착맨 (zilioner)> ^^7\n',
 '[10:33:31] <jujitsuu111> 침 하\n',
 '[10:33:34] <ssurebba> 침하\n',
 '[10:33:35] <k01980> 침하\n',
 '[10:33:35] <jujitsuu111> 오늘 좆구 합니까?\n',
 '[10:33:38] <ssurebba> ??\n',
 '[10:33:39] <zudol0104> 띠용?\n',
 '[10:33:39] <%+devol3851> 침하\n',
 '[10:33:39] <중1이 (rlqkwkwk2)> ㅊㅎ\n',
 '[10:33:40] <%$macaupack> 침하침하\n',
 '[10:33:40] <motosjw> 와 류방\n',
 '[10:33:41] <20170823전역 (cktmddn1125)> 저게임 하시게뇨?\n',
 '[10:33:41] <칼리야 (agility8910)> 침하\n',
 '[10:33:42] <%연결리스트 (cpp0094)> 침하\n',
 '[10:33:43] <하얗다 (krea501800)> 침하\n',
 '[10:33:44] <라면은물적게 (paenin)> 침하\n',
 '[10:33:45] <gkadhkd> 치하\n',
 '[10:33:45] <간다가 (77ysb)> 침하\n',
 '[10:33:45] <레바왼쪽유두 (parkgane1)> ㅎㅇ\n',
 '[10:33:46] <앜앜앜앜앜앜앜 (hsm3559)> 치마\n',
 '[10:33:46] <motosjw> ㅊㅎ\n',
 '[10:33:46] <애꽃송 (ghji0402)> 침하\n',
 '[10:33:46] <dydnzl> 치마\n',
 '[10:33:47] <%시공의아들 (ehdals24)> 5일간 없는줄알았는데 오늘갑

In [250]:
def preprocessing_chat_tolist(chat, num = 3) : 
    """ 
        chat : chat에는 채팅로그 넣기
        num : 그룹 선택 / 안주면 기본값 3 (0 = 전체, 1 = 시간(방송기준) 2 = ID 3 = 채팅)
    """
    import re
    text = chat
    my = re.compile('\[([0-9:]*)\] <(\S*[ ]*\S*)> (\w.*)')
    word_tolist = []
    for line in text:
        mytext = my.search(line)
        if mytext :
            word_tolist.append([mytext.group(num)])
    return word_tolist

In [251]:
text_list = preprocessing_chat_tolist(text)

In [252]:
def tokenize_tolist(word_list_tolist) : 
    """
        word_list_tolist : 전처리, 리스트화 된 채팅 로그 (preprocessing_tolist 리턴값)
    """
    start_time = time.time()
    tokens_ko_tolist = []
    line_list = []
    for word_list in word_tolist : 
        for word in word_list : 
            for i in range(len(ko.pos(word))) : 
                line_list.append(ko.pos(word)[i][0])
                if i+1 == len(ko.pos(word)) : 
                    tokens_ko_tolist.append(line_list)
                    line_list = []
    end_time = time.time()
    print(end_time - start_time)
    return tokens_ko_tolist

In [253]:
def matching(word_tolist, tokens_ko_tolist) : 
    """
        word_list : 전처리, 리스트화 된 채팅 로그 (preprocessing_tolist 리턴값)
    """
    matching_list = []
    for i , value in enumerate(word_tolist):
        try : 
            for target in tokens_ko_tolist[i]:
                if target not in value[0]:
                    matching_list.append((target, value[0]))
        except Exception as e : 
            print(e, target, value)
    return matching_list

In [254]:
text_array = np.array(text_list)

In [255]:
def flatten_list(word_tolist) : 
    text_array = np.array(text_list)
    text_array_flat = text_array.flatten()
    return text_array_flat

In [256]:
def tokenize_tolist(word_list_tolist) :
    tokens_ko_tolist = []
    line_list = []
    for word in text_array_flat : 
        for i in range(len(ko.pos(word))) : 
            line_list.append(ko.pos(word)[i][0])
            if i+1 == len(ko.pos(word)) : 
                tokens_ko_tolist.append(line_list)
                line_list = []
    return tokens_ko_tolist

In [257]:
text_array_flat = flatten_list(text_list)

In [259]:
tokens_ko_tolist = tokenize_tolist(text_array_flat)
tokens_ko_tolist

[['치뫄'],
 ['하이', '요'],
 ['침', '하'],
 ['침하'],
 ['침하'],
 ['오늘', '좆', '이', '구', '하', 'ㅂ니까', '?'],
 ['띠', '용', '?'],
 ['침하'],
 ['ㅊㅎ'],
 ['침하', '침하'],
 ['오', '아', '류', '바', 'ㅇ'],
 ['저', '게임', '하', '시', '게', '뇨', '?'],
 ['침하'],
 ['침하'],
 ['침하'],
 ['침하'],
 ['치하'],
 ['침하'],
 ['ㅎㅇ'],
 ['치마'],
 ['ㅊㅎ'],
 ['침하'],
 ['치마'],
 ['5', '일', '간', '없', '는', '줄', '알', '았', '는데', '오늘', '갑자기', '하', '시', '네'],
 ['침하', '~'],
 ['침하', '침하'],
 ['치마'],
 ['침하'],
 ['침하'],
 ['돌겜하세넹'],
 ['침하'],
 ['아키', '침하'],
 ['침', '화'],
 ['아', '좆스스톤;'],
 ['침하'],
 ['방송', '업', '타임', ':', '0', '시간', '01', '분', '56', '초'],
 ['침하'],
 ['침하ㅡ'],
 ['침', '부', 'ㅇ', '님', 'ㅎ2'],
 ['오늘', '방송', '안', '하', '자', '나', '요'],
 ['치마'],
 ['침하'],
 ['ㅎㅎ'],
 ['ㅠ', 'ㅠ', 'ㅠ', 'ㅠ', 'ㅠ', 'ㅠ', 'ㅠ', 'ㅠ', 'ㅠ', '호', '다다', 'ㄱ'],
 ['침하'],
 ['침하'],
 ['침하'],
 ['침', '바'],
 ['치마'],
 ['침하'],
 ['맨날', '하스스톤이냐', '이', '말', '이야'],
 ['광고', '개', '기', '네', 'ㅡㅡ'],
 ['침하'],
 ['오늘', '방송', '없', '다더니'],
 ['침하', '!!'],
 ['침하'],
 ['침하', '애', 'ㄴ', '메리 크리스마스', '~', '~', '!'],
 ['치마', '치마'],

In [266]:
%time tokenize_tolist(text_array_flat)

CPU times: user 2min 3s, sys: 170 ms, total: 2min 3s
Wall time: 2min 3s


[['치뫄'],
 ['하이', '요'],
 ['침', '하'],
 ['침하'],
 ['침하'],
 ['오늘', '좆', '이', '구', '하', 'ㅂ니까', '?'],
 ['띠', '용', '?'],
 ['침하'],
 ['ㅊㅎ'],
 ['침하', '침하'],
 ['오', '아', '류', '바', 'ㅇ'],
 ['저', '게임', '하', '시', '게', '뇨', '?'],
 ['침하'],
 ['침하'],
 ['침하'],
 ['침하'],
 ['치하'],
 ['침하'],
 ['ㅎㅇ'],
 ['치마'],
 ['ㅊㅎ'],
 ['침하'],
 ['치마'],
 ['5', '일', '간', '없', '는', '줄', '알', '았', '는데', '오늘', '갑자기', '하', '시', '네'],
 ['침하', '~'],
 ['침하', '침하'],
 ['치마'],
 ['침하'],
 ['침하'],
 ['돌겜하세넹'],
 ['침하'],
 ['아키', '침하'],
 ['침', '화'],
 ['아', '좆스스톤;'],
 ['침하'],
 ['방송', '업', '타임', ':', '0', '시간', '01', '분', '56', '초'],
 ['침하'],
 ['침하ㅡ'],
 ['침', '부', 'ㅇ', '님', 'ㅎ2'],
 ['오늘', '방송', '안', '하', '자', '나', '요'],
 ['치마'],
 ['침하'],
 ['ㅎㅎ'],
 ['ㅠ', 'ㅠ', 'ㅠ', 'ㅠ', 'ㅠ', 'ㅠ', 'ㅠ', 'ㅠ', 'ㅠ', '호', '다다', 'ㄱ'],
 ['침하'],
 ['침하'],
 ['침하'],
 ['침', '바'],
 ['치마'],
 ['침하'],
 ['맨날', '하스스톤이냐', '이', '말', '이야'],
 ['광고', '개', '기', '네', 'ㅡㅡ'],
 ['침하'],
 ['오늘', '방송', '없', '다더니'],
 ['침하', '!!'],
 ['침하'],
 ['침하', '애', 'ㄴ', '메리 크리스마스', '~', '~', '!'],
 ['치마', '치마'],

In [108]:
def matching(word_tolist, tokens_ko_tolist) : 
    """
        word_list : 전처리, 리스트화 된 채팅 로그 (preprocessing_tolist 리턴값)
    """
    matching_list = []
    for i , value in enumerate(word_tolist):
        try : 
            for target in tokens_ko_tolist[i]:
                if target not in value:
                    matching_list.append((tokens_ko_tolist[i], value))
        except Exception as e : 
            print(e, target, value)
    return matching_list

In [109]:
match = matching(text_array_flat, tokens_ko_tolist )
match

[(['오늘', '좆', '이', '구', '하', 'ㅂ니까', '?'], '오늘 좆구 합니까?'),
 (['오늘', '좆', '이', '구', '하', 'ㅂ니까', '?'], '오늘 좆구 합니까?'),
 (['오늘', '좆', '이', '구', '하', 'ㅂ니까', '?'], '오늘 좆구 합니까?'),
 (['오', '아', '류', '바', 'ㅇ'], '와 류방'),
 (['오', '아', '류', '바', 'ㅇ'], '와 류방'),
 (['오', '아', '류', '바', 'ㅇ'], '와 류방'),
 (['오', '아', '류', '바', 'ㅇ'], '와 류방'),
 (['침', '부', 'ㅇ', '님', 'ㅎ2'], '침붕님 ㅎ2'),
 (['침', '부', 'ㅇ', '님', 'ㅎ2'], '침붕님 ㅎ2'),
 (['ㅠ', 'ㅠ', 'ㅠ', 'ㅠ', 'ㅠ', 'ㅠ', 'ㅠ', 'ㅠ', 'ㅠ', '호', '다다', 'ㄱ'],
  'ㅠㅠㅠㅠㅠㅠㅠㅠㅠ호다닥'),
 (['ㅠ', 'ㅠ', 'ㅠ', 'ㅠ', 'ㅠ', 'ㅠ', 'ㅠ', 'ㅠ', 'ㅠ', '호', '다다', 'ㄱ'],
  'ㅠㅠㅠㅠㅠㅠㅠㅠㅠ호다닥'),
 (['침하', '애', 'ㄴ', '메리 크리스마스', '~', '~', '!'], '침하 앤 메리 크리스마스 ~~!'),
 (['침하', '애', 'ㄴ', '메리 크리스마스', '~', '~', '!'], '침하 앤 메리 크리스마스 ~~!'),
 (['감자', '방', '에서', '왜', '그러', '시', '었', '어요'], '감자방에서 왜그러셨어요'),
 (['감자', '방', '에서', '왜', '그러', '시', '었', '어요'], '감자방에서 왜그러셨어요'),
 (['허스', '스마', 'ㄴ', '하', '아라'], '허스 스만해라'),
 (['허스', '스마', 'ㄴ', '하', '아라'], '허스 스만해라'),
 (['허스', '스마', 'ㄴ', '하', '아라'], '허스 스만해라'),
 (['허스', '스마', 'ㄴ', '하', '아

In [112]:
tokens_ko_tolist

[['치뫄'],
 ['하이', '요'],
 ['침', '하'],
 ['침하'],
 ['침하'],
 ['오늘', '좆', '이', '구', '하', 'ㅂ니까', '?'],
 ['띠', '용', '?'],
 ['침하'],
 ['ㅊㅎ'],
 ['침하', '침하'],
 ['오', '아', '류', '바', 'ㅇ'],
 ['저', '게임', '하', '시', '게', '뇨', '?'],
 ['침하'],
 ['침하'],
 ['침하'],
 ['침하'],
 ['치하'],
 ['침하'],
 ['ㅎㅇ'],
 ['치마'],
 ['ㅊㅎ'],
 ['침하'],
 ['치마'],
 ['5', '일', '간', '없', '는', '줄', '알', '았', '는데', '오늘', '갑자기', '하', '시', '네'],
 ['침하', '~'],
 ['침하', '침하'],
 ['치마'],
 ['침하'],
 ['침하'],
 ['돌겜하세넹'],
 ['침하'],
 ['아키', '침하'],
 ['침', '화'],
 ['아', '좆스스톤;'],
 ['침하'],
 ['방송', '업', '타임', ':', '0', '시간', '01', '분', '56', '초'],
 ['침하'],
 ['침하ㅡ'],
 ['침', '부', 'ㅇ', '님', 'ㅎ2'],
 ['오늘', '방송', '안', '하', '자', '나', '요'],
 ['치마'],
 ['침하'],
 ['ㅎㅎ'],
 ['ㅠ', 'ㅠ', 'ㅠ', 'ㅠ', 'ㅠ', 'ㅠ', 'ㅠ', 'ㅠ', 'ㅠ', '호', '다다', 'ㄱ'],
 ['침하'],
 ['침하'],
 ['침하'],
 ['침', '바'],
 ['치마'],
 ['침하'],
 ['맨날', '하스스톤이냐', '이', '말', '이야'],
 ['광고', '개', '기', '네', 'ㅡㅡ'],
 ['침하'],
 ['오늘', '방송', '없', '다더니'],
 ['침하', '!!'],
 ['침하'],
 ['침하', '애', 'ㄴ', '메리 크리스마스', '~', '~', '!'],
 ['치마', '치마'],

In [113]:
match

[(['오늘', '좆', '이', '구', '하', 'ㅂ니까', '?'], '오늘 좆구 합니까?'),
 (['오늘', '좆', '이', '구', '하', 'ㅂ니까', '?'], '오늘 좆구 합니까?'),
 (['오늘', '좆', '이', '구', '하', 'ㅂ니까', '?'], '오늘 좆구 합니까?'),
 (['오', '아', '류', '바', 'ㅇ'], '와 류방'),
 (['오', '아', '류', '바', 'ㅇ'], '와 류방'),
 (['오', '아', '류', '바', 'ㅇ'], '와 류방'),
 (['오', '아', '류', '바', 'ㅇ'], '와 류방'),
 (['침', '부', 'ㅇ', '님', 'ㅎ2'], '침붕님 ㅎ2'),
 (['침', '부', 'ㅇ', '님', 'ㅎ2'], '침붕님 ㅎ2'),
 (['ㅠ', 'ㅠ', 'ㅠ', 'ㅠ', 'ㅠ', 'ㅠ', 'ㅠ', 'ㅠ', 'ㅠ', '호', '다다', 'ㄱ'],
  'ㅠㅠㅠㅠㅠㅠㅠㅠㅠ호다닥'),
 (['ㅠ', 'ㅠ', 'ㅠ', 'ㅠ', 'ㅠ', 'ㅠ', 'ㅠ', 'ㅠ', 'ㅠ', '호', '다다', 'ㄱ'],
  'ㅠㅠㅠㅠㅠㅠㅠㅠㅠ호다닥'),
 (['침하', '애', 'ㄴ', '메리 크리스마스', '~', '~', '!'], '침하 앤 메리 크리스마스 ~~!'),
 (['침하', '애', 'ㄴ', '메리 크리스마스', '~', '~', '!'], '침하 앤 메리 크리스마스 ~~!'),
 (['감자', '방', '에서', '왜', '그러', '시', '었', '어요'], '감자방에서 왜그러셨어요'),
 (['감자', '방', '에서', '왜', '그러', '시', '었', '어요'], '감자방에서 왜그러셨어요'),
 (['허스', '스마', 'ㄴ', '하', '아라'], '허스 스만해라'),
 (['허스', '스마', 'ㄴ', '하', '아라'], '허스 스만해라'),
 (['허스', '스마', 'ㄴ', '하', '아라'], '허스 스만해라'),
 (['허스', '스마', 'ㄴ', '하', '아

In [114]:
def to_array(matching_list) : 
    match_arr = np.array(matching_list)
    frame = pd.DataFrame(match_arr)
    return frame

In [115]:
match_arr = np.array(match)

In [116]:
frame = pd.DataFrame(match_arr)

In [117]:
frame

,0,1
0,"[오늘, 좆, 이, 구, 하, ㅂ니까, ?]",오늘 좆구 합니까?
1,"[오늘, 좆, 이, 구, 하, ㅂ니까, ?]",오늘 좆구 합니까?
2,"[오늘, 좆, 이, 구, 하, ㅂ니까, ?]",오늘 좆구 합니까?
3,"[오, 아, 류, 바, ㅇ]",와 류방
4,"[오, 아, 류, 바, ㅇ]",와 류방
5,"[오, 아, 류, 바, ㅇ]",와 류방
6,"[오, 아, 류, 바, ㅇ]",와 류방
7,"[침, 부, ㅇ, 님, ㅎ2]",침붕님 ㅎ2
8,"[침, 부, ㅇ, 님, ㅎ2]",침붕님 ㅎ2
9,"[ㅠ, ㅠ, ㅠ, ㅠ, ㅠ, ㅠ, ㅠ, ㅠ, ㅠ, 호, 다다, ㄱ]",ㅠㅠㅠㅠㅠㅠㅠㅠㅠ호다닥


In [118]:
frame = frame.drop_duplicates([1])

In [119]:
frame = frame.reset_index()

In [120]:
frame = frame.drop('index',axis=1)

In [121]:
frame = frame.rename(columns={0:'morphs', 1:'origin'})

In [122]:
frame_backup = frame

In [123]:
frame_backup

,morphs,origin
0,"[오늘, 좆, 이, 구, 하, ㅂ니까, ?]",오늘 좆구 합니까?
1,"[오, 아, 류, 바, ㅇ]",와 류방
2,"[침, 부, ㅇ, 님, ㅎ2]",침붕님 ㅎ2
3,"[ㅠ, ㅠ, ㅠ, ㅠ, ㅠ, ㅠ, ㅠ, ㅠ, ㅠ, 호, 다다, ㄱ]",ㅠㅠㅠㅠㅠㅠㅠㅠㅠ호다닥
4,"[침하, 애, ㄴ, 메리 크리스마스, ~, ~, !]",침하 앤 메리 크리스마스 ~~!
5,"[감자, 방, 에서, 왜, 그러, 시, 었, 어요]",감자방에서 왜그러셨어요
6,"[허스, 스마, ㄴ, 하, 아라]",허스 스만해라
7,"[왤케, 일찍, 방송, 하, 아요, 직장, 에서, 몰래, 보, 기, 눈치, 보이, ...",왤케 일찍 방송해요 직장에서 몰래보기 눈치보이자너
8,"[하스, 구마, ㄴ, 해]",하스 구만해
9,"[총, 번개, 악마, 드래곤, 물, 바람, 보자기, 스포, ㄴ, 지, 늑대, 나무,...",총번개악마드래곤물바람보자기스폰지늑대나무사람뱀가위불주먹 게임 ;


In [124]:
frame

,morphs,origin
0,"[오늘, 좆, 이, 구, 하, ㅂ니까, ?]",오늘 좆구 합니까?
1,"[오, 아, 류, 바, ㅇ]",와 류방
2,"[침, 부, ㅇ, 님, ㅎ2]",침붕님 ㅎ2
3,"[ㅠ, ㅠ, ㅠ, ㅠ, ㅠ, ㅠ, ㅠ, ㅠ, ㅠ, 호, 다다, ㄱ]",ㅠㅠㅠㅠㅠㅠㅠㅠㅠ호다닥
4,"[침하, 애, ㄴ, 메리 크리스마스, ~, ~, !]",침하 앤 메리 크리스마스 ~~!
5,"[감자, 방, 에서, 왜, 그러, 시, 었, 어요]",감자방에서 왜그러셨어요
6,"[허스, 스마, ㄴ, 하, 아라]",허스 스만해라
7,"[왤케, 일찍, 방송, 하, 아요, 직장, 에서, 몰래, 보, 기, 눈치, 보이, ...",왤케 일찍 방송해요 직장에서 몰래보기 눈치보이자너
8,"[하스, 구마, ㄴ, 해]",하스 구만해
9,"[총, 번개, 악마, 드래곤, 물, 바람, 보자기, 스포, ㄴ, 지, 늑대, 나무,...",총번개악마드래곤물바람보자기스폰지늑대나무사람뱀가위불주먹 게임 ;


In [40]:
frame.to_csv('test.csv', sep = '|', mode = 'w')

In [123]:
for wd in test1 : 
    if wd in test2 : 
        target_index = test2.index(wd)
#         print(target_index)
        remove_word = test2[target_index : target_index + len(wd)]
        test2 = test2.replace(remove_word, "")
        print(test2)

끝났어요 데이트하러가셈
끝났 데이트하러가셈
끝났 하러가셈
끝났 러가셈
끝났 가셈
끝났 셈
끝났 


In [46]:
# miss = []
# for i in range(len(frame)) : 
#     try : 
#         for j, wd in enumerate(frame[0][i]) : 
#             if wd in frame[1][i] : 
#                 target_index = frame[1][i].index(wd)
#                 remove_word = frame[1][i][target_index : target_index + len(wd)]
#                 frame_backup[1][i] = frame[1][i].replace(remove_word, "")
# #                 if j+1 == len(frame[0][i]) : 
# #                     miss.append(frame[1][i])
#     except Exception as e : 
#         print(e)

In [125]:
def remove_word(x):
    try : 
        for wd1 in frame[0]:
            for wd in wd1:
                if wd in x : 
                    target_index = x.index(wd)
                    remove_word = x[target_index: target_index + len(wd)]
                    return x.replace(remove_word, "")
    except Exception as e : print(e)

# frame[2] = frame[1].apply(remove_word)


In [126]:
miss = []
for i in range(len(frame)) : 
    try : 
        for j, wd in enumerate(frame[0][i]) : 
            if wd in frame[1][i] : 
                target_index = frame[1][i].index(wd)
                remove_word = frame[1][i][target_index : target_index + len(wd)]
                frame_backup[1][i] = frame[1][i].replace(remove_word, "")
#                 if j+1 == len(frame[0][i]) : 
#                     miss.append(frame[1][i])
    except Exception as e : 
        print(e)

0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0


0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0


0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0


In [ ]:
for wd in test1 : 
    if wd in test2 : 
        target_index = test2.index(wd)
#         print(target_index)
        remove_word = test2[target_index : target_index + len(wd)]
        test2 = test2.replace(remove_word, "")
        print(test2)

In [ ]:
miss = []
for j, wd in enumerate(frame[0][i]) : 
    if wd in frame[1][i] : 
        target_index = frame[1][i].index(wd)
        remove_word = frame[1][i][target_index : target_index + len(wd)]
        frame_backup[1][i] = frame[1][i].replace(remove_word, "")
#                 if j+1 == len(frame[0][i]) : 
#                     miss.append(frame[1][i])
    except Exception as e : 
        print(e)

In [130]:
frame['morphs'][frame['origin']== '왤케 일찍 방송해요 직장에서 몰래보기 눈치보이자너']

7    [왤케, 일찍, 방송, 하, 아요, 직장, 에서, 몰래, 보, 기, 눈치, 보이, ...
Name: morphs, dtype: object

In [173]:
frame['origin'][:10].apply(lambda x : x)

0                            오늘 좆구 합니까?
1                                  와 류방
2                                침붕님 ㅎ2
3                          ㅠㅠㅠㅠㅠㅠㅠㅠㅠ호다닥
4                     침하 앤 메리 크리스마스 ~~!
5                          감자방에서 왜그러셨어요
6                               허스 스만해라
7           왤케 일찍 방송해요 직장에서 몰래보기 눈치보이자너
8                                하스 구만해
9    총번개악마드래곤물바람보자기스폰지늑대나무사람뱀가위불주먹 게임 ;
Name: origin, dtype: object

In [207]:
a = iter(frame['morphs'][frame['origin']== '오늘 좆구 합니까?'])
for i in next(a) : 
    print(i)

오늘
좆
이
구
하
ㅂ니까
?


In [209]:
for i in frame['morphs'][frame['origin']== '오늘 좆구 합니까?'] : 
    print(i)

['오늘', '좆', '이', '구', '하', 'ㅂ니까', '?']


In [234]:
def remove_word(x):
    line = iter(frame['morphs'][frame['origin']== x])
    new_line = next(line)
    for wd in (new_line) :
        if wd in x : 
            target_index = x.index(wd)
            remove_word = x[target_index: target_index + len(wd)]
            x = x.replace(remove_word, "")
    return x.strip().split(' ')

In [235]:
frame['cathch'] = frame['origin'].apply(lambda x : remove_word(x))

In [236]:
frame

,morphs,origin,cathch
0,"[오늘, 좆, 이, 구, 하, ㅂ니까, ?]",오늘 좆구 합니까?,[합니까]
1,"[오, 아, 류, 바, ㅇ]",와 류방,"[와, 방]"
2,"[침, 부, ㅇ, 님, ㅎ2]",침붕님 ㅎ2,[붕]
3,"[ㅠ, ㅠ, ㅠ, ㅠ, ㅠ, ㅠ, ㅠ, ㅠ, ㅠ, 호, 다다, ㄱ]",ㅠㅠㅠㅠㅠㅠㅠㅠㅠ호다닥,[다닥]
4,"[침하, 애, ㄴ, 메리 크리스마스, ~, ~, !]",침하 앤 메리 크리스마스 ~~!,[앤]
5,"[감자, 방, 에서, 왜, 그러, 시, 었, 어요]",감자방에서 왜그러셨어요,[셨]
6,"[허스, 스마, ㄴ, 하, 아라]",허스 스만해라,[스만해라]
7,"[왤케, 일찍, 방송, 하, 아요, 직장, 에서, 몰래, 보, 기, 눈치, 보이, ...",왤케 일찍 방송해요 직장에서 몰래보기 눈치보이자너,"[해요, , , 이]"
8,"[하스, 구마, ㄴ, 해]",하스 구만해,[구만]
9,"[총, 번개, 악마, 드래곤, 물, 바람, 보자기, 스포, ㄴ, 지, 늑대, 나무,...",총번개악마드래곤물바람보자기스폰지늑대나무사람뱀가위불주먹 게임 ;,[스폰]


In [ ]:
miss = []

def lab() : 
    for j, wd in enumerate(frame[0][i]) : 
        if wd in frame[1][i] : 
            target_index = frame[1][i].index(wd)
            remove_word = frame[1][i][target_index : target_index + len(wd)]
            frame_backup[1][i] = frame[1][i].replace(remove_word, "")


In [89]:
frame = pd.read_csv('../new_matching_words/#beyou0728/2018-12-04_#beyou0728.log.csv', sep='|')

In [ ]:
def remove_word(x):
    try : 
        for i,wd1 in enumerate(frame['morphs']):
            for wd in wd1:
                if wd in frame['origin'][i] : 
                    target_index = x.index(wd)
                    remove_word = x[target_index: target_index + len(wd)]
                    return x.replace(remove_word, "")
    except Exception as e : print(e)

# frame[2] = frame[1].apply(remove_word)


In [ ]:
# frame['check'] = 0
# miss = []
# for i in range(len(frame)) : 
#     try : 
#         for j, wd in enumerate(frame['morphs'][i]) : 
#             if wd in frame['origin'][i] : 
#                 target_index = frame['origin'][i].index(wd)
#                 remove_word = frame['origin'][i][target_index : target_index + len(wd)]
#                 frame['check'][i] = frame['origin'][i].replace(remove_word, "")

#     except Exception as e : 
#         print(e)

/root/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/root/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
